# UFRN Analytics

### Análise sobre os dados de Extensão

- Bancos de dados relacionados a [<b>Extensão</b>](http://dados.ufrn.br/group/extensao) no Portal de Dados Abertos da UFRN:  

[<b>Produtos de Extensão:</b>](http://dados.ufrn.br/dataset/produtos-de-extensao) Relação de produtos de extensão gerados pela UFRN  
[<b>Atividades de Extensão:</b>](http://dados.ufrn.br/dataset/atividades-de-extensao) Relação das atividades de extensão da UFRN

- Outros bancos de dados relevantes:

[<b>Docentes:</b>](http://dados.ufrn.br/dataset/docentes/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070) Docentes da UFRN

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import re
from datetime import datetime as dt
%matplotlib inline

# Produtos de Extensão

A [Resolução n 077/2017 - CONSEPE/UFRN](https://sigrh.ufrn.br/sigrh/downloadArquivo?idArquivo=4186769&key=7725dd34a0808e1e0cf2f8d29894d8d3), que dispõe sobre as modalidades e ações de extensão universitária na Universidade Federal do Rio Grande do Norte, estabelece:

<i>Art. 17. Os produtos acadêmicos caracterizam-se por serem decorrentes das ações de extensão, ensino e pesquisa para difusão e divulgação artística, cultural, científica ou tecnológica.  
Parágrafo único. Os produtos são caracterizados por livros, anais, artigos, textos, revistas, manuais, cartilhas, jornais e relatórios, materiais didáticos, vídeos, filmes, programas de rádio e TV, softwares, jogos, modelos didáticos, partituras, arranjos musicais, peças teatrais, mídias informacionais, performances artísticas dentre outros.</i>

In [3]:
# import dataset
produtos = pd.read_csv('produtos-de-extensao.csv', delimiter = ";")
# drop unnecessary column, previously detected
produtos.drop(['Unnamed: 16'], axis=1, inplace=True);
# show the first line
produtos.head(1)

,titulo,produto_gerado,tiragem,situacao,tipo_projeto,tipo_produto,ano,data_inicio,data_fim,unidade,forma_projeto,area_principal,area_conhecimento_cnpq,quantidade_discente,publico_estimado,bolsas_concedidas
0,3ª edição da Revista Transgressões: Ciências C...,NaN,1,CONCLUÍDA,PRODUTO,REVISTA,2014,2014-01-01,2014-05-20,DEPARTAMENTO DE DIREITO PUBLICO,INTERNO,DIREITOS HUMANOS E JUSTIÇA,Ciências Sociais Aplicadas,18,50,0


In [4]:
print("Número de linhas: ", produtos.shape[0])
print("Número de colunas: ", produtos.shape[1])

Número de linhas:  335
Número de colunas:  16


In [5]:
produtos.columns

Index(['titulo', 'produto_gerado', 'tiragem', 'situacao', 'tipo_projeto',
       'tipo_produto', 'ano', 'data_inicio', 'data_fim', 'unidade',
       'forma_projeto', 'area_principal', 'area_conhecimento_cnpq',
       'quantidade_discente', 'publico_estimado', 'bolsas_concedidas'],
      dtype='object')

O dataset possui 16 colunas. Apesar das descrições presentes em seu [dicionário de dados](http://dados.ufrn.br/dataset/produtos-de-extensao/resource/a393e7ab-fcdb-4522-978b-f954b2348ca1), uma análise dos valores presentes em cada coluna nos permite analisar a consistência dos dados.  
Por exemplo, através da análise dos valores presentes na coluna <b>'ano'</b>, descobrimos que o dataset possui registros de Produtos de Extensão concebidos entre 2008 e 2017.

In [6]:
produtos['ano'].value_counts().sort_index()

2008    29
2009    22
2010    51
2011    38
2012    36
2013    22
2014    22
2015    36
2016    40
2017    39
Name: ano, dtype: int64

Há produtos "EM EXECUÇÃO", mas a maioria deles tem status "CONCLUÍDO" e isso pode ser observado através da coluna <b>'situacao'</b>.  
Através da coluna <b>'forma_projeto'</b>, vemos que todos os produtos são oriundos de projetos internos.

In [7]:
produtos['situacao'].value_counts()

CONCLUÍDA      302
EM EXECUÇÃO     33
Name: situacao, dtype: int64

In [8]:
produtos['forma_projeto'].value_counts()

INTERNO    335
Name: forma_projeto, dtype: int64

Como visto no trecho da resolução colocado acima, produtos de extensão tem natureza diversa.  
A coluna <b>'tipo_produto'</b> nos permite listar os tipos de produtos presentes no dataset.

In [9]:
produtos['tipo_produto'].value_counts()

OUTROS                          60
ARTIGO                          43
PRODUTO AUDIOVISUAL - VÍDEO     41
LIVRO                           38
RELATÓRIO TÉCNICO               26
REVISTA                         24
CARTILHA                        23
MANUAL                          19
APLICATIVO PARA COMPUTADOR      12
COMUNICAÇÃO                      8
PRODUTO AUDIOVISUAL - DVD        7
PRODUTO ARTÍSTICO                7
PROGRAMA DE TV                   6
JORNAL                           5
PRODUTO AUDIOVISUAL - OUTROS     4
PRODUTO AUDIOVISUAL - CD         3
PROGRAMA DE RÁDIO                3
PRODUTO AUDIOVISUAL - FILME      3
JOGO EDUCATIVO                   2
ANAIS                            1
Name: tipo_produto, dtype: int64

A Universidade também pode conceder bolsas de extensão para a criação de produtos de extensão.  
Mais de 90% dos produtos é feito sem a concessão de bolsas. Cerca de 8% dos produtos de extensão registrados contou com ao menos uma bolsa de extensão. O número máximo de bolsas concedidas por produto é 2 e isso aconteceu em cerca de 2% dos casos.

In [10]:
produtos['bolsas_concedidas'].value_counts(normalize=True)

0    0.916418
1    0.059701
2    0.023881
Name: bolsas_concedidas, dtype: float64

Após uma primeira análise exploratória das colunas do dataset, pode-se passar à utilização de ferramentas de visualização que nos ajudem a compreender com mais profundidade 

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Bar(
    x=produtos['ano'].value_counts().sort_index().index,
    y=produtos['ano'].value_counts().sort_index().values,
    text=(produtos['ano'].value_counts(normalize=True)*100).sort_index().values,
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Produtos de Extensão 2008 - 2017',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='produtos_extensao_ano')

In [12]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": produtos['tipo_produto'].value_counts().values,
      "labels": produtos['tipo_produto'].value_counts().index,
      "domain": {"x": [0, .48]},
      "name": "Tipo",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Produtos de Extensão 2008 - 2017",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Produtos",
                "x": 0.20,
                "y": 0.5
            }
        ],
      'showlegend': False
    }
}
py.iplot(fig, filename='produtos_extensao_tipo')

Outra análise viável diz respeito à diferença de tempo entre as datas de início e término de cada produto.

In [13]:
produtos['data_inicio'] = pd.to_datetime(produtos['data_inicio'])
produtos['data_fim'] = pd.to_datetime(produtos['data_fim'])
duracao = (produtos['data_fim'] - produtos['data_inicio'])

In [14]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Histogram(
    x=duracao.dt.days,
    opacity=0.6
)

data = [trace]
layout = go.Layout(
    title='Número de Dias para Criação de Produtos de Extensão 2008 - 2017',
    xaxis=dict(
        title='Número de Dias'
    ),
    yaxis=dict(
        title='Quantidade de Produtos'
    ),
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='produtos_dias')

# Atividades de Extensão

O dataset dedicado às atividades de extensão possui um número maior e mais diverso de registros. 

De acordo com a [Resolução n 077/2017 - CONSEPE/UFRN](https://sigrh.ufrn.br/sigrh/downloadArquivo?idArquivo=4186769&key=7725dd34a0808e1e0cf2f8d29894d8d3), são consideradas ações de extensão universitária: cursos, eventos, produtos, prestação de serviços. A resolução também estabelece que as ações deverão ser formalizadas por meio de projetos e programas, que constituem as modalidades de extensão na UFRN.  

Vale salientar que a resolução mencionada acima entrou em vigor em 27 de junho de 2017, substituindo a [Resolução n 053/2008 - CONSEPE/UFRN](https://sigrh.ufrn.br/sigrh/downloadArquivo?idArquivo=78156&key=19550480c9323a62d0e0d4a1aa99c4df).  
No documento anterior, que vigorava durante o registro da maioria absoluta dos dados presentes no dataset, considerava-se como modalidades das ações de extensão: programas, projetos, cursos, eventos, produtos, prestação de serviços.

In [54]:
# import dataset
atividades = pd.read_csv('atividades-de-extensao.csv', delimiter = ";")
# drop some unnecessary columns and lines
atividades = atividades[atividades['tipo_projeto']=='EXTENSÃO']
atividades.drop(['Unnamed: 16', 'Unnamed: 17'], axis=1, inplace=True)
# correct this line
atividades.loc[atividades['data_fim']=='3016-12-18','data_fim'] = '2016-12-18'
atividades.loc[atividades['data_fim']=='2107-12-31','data_fim'] = '2017-12-31'
# show the first line
atividades.head(1)

,ano,tipo_atividade,titulo,data_inicio,data_fim,coordenador,area_tematica,situacao,tipo_projeto,bolsas_solicitadas,bolsas_concedidas,publico_interno,bolsas_edital,numero_edital,edital,municipio
0,2017,CURSO,Arduino Básico,2017-05-02,2017-05-04,JOSE HENRIQUE FERNANDEZ,TECNOLOGIA E PRODUÇÃO,AGUARDANDO APROVAÇÃO DOS DEPARTAMENTOS,EXTENSÃO,0,0,30,NaN,NaN,NaN,NaN


In [16]:
print("Número de linhas: ", atividades.shape[0])
print("Número de colunas: ", atividades.shape[1])

Número de linhas:  8083
Número de colunas:  16


In [17]:
atividades.columns

Index(['ano', 'tipo_atividade', 'titulo', 'data_inicio', 'data_fim',
       'coordenador', 'area_tematica', 'situacao', 'tipo_projeto',
       'bolsas_solicitadas', 'bolsas_concedidas', 'publico_interno',
       'bolsas_edital', 'numero_edital', 'edital', 'municipio'],
      dtype='object')

In [18]:
atividades['ano'].value_counts().sort_index()

2014    1806
2015    2332
2016    2231
2017    1714
Name: ano, dtype: int64

In [27]:
atividades['situacao'].value_counts().sort_index()

AGUARDANDO APROVAÇÃO DOS DEPARTAMENTOS           102
AGUARDANDO AVALIAÇÃO                               1
ANALISANDO SOLICITAÇÃO DE RECONSIDERAÇÃO           4
APROVADO COM RECURSOS                             84
APROVADO SEM RECURSOS                            467
CONCLUÍDA                                       5305
EM AVALIAÇÃO                                      28
EM EXECUÇÃO                                      842
NÃO APROVADA                                     334
NÃO APROVADO PELOS DEPARTAMENTOS                  16
PENDENTE DE RELATÓRIO                            624
PROJETO CANCELADO                                249
PROPOSTA DEVOLVIDA PARA COORDENADOR REEDITAR       1
SUBMETIDA                                         26
Name: situacao, dtype: int64

In [28]:
atividades['tipo_atividade'].value_counts().sort_index()

CURSO       1215
EVENTO      3387
PRODUTO      147
PROGRAMA      73
PROJETO     3261
Name: tipo_atividade, dtype: int64

In [56]:
atividades['data_inicio'] = pd.to_datetime(atividades['data_inicio'])
atividades['data_fim'] = pd.to_datetime(atividades['data_fim'])

In [41]:
atividades['data_inicio'].dt.month.value_counts().sort_index()

1      446
2      817
3     1841
4      617
5      751
6      492
7      355
8      731
9      611
10     573
11     586
12     263
Name: data_inicio, dtype: int64

In [68]:
import plotly.plotly as py
import plotly.graph_objs as go

x = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

trace1 = go.Bar(
    name='2014',
    x=x,
    y=atividades.loc[atividades['ano']=='2014', 'data_inicio'].dt.month.value_counts().sort_index(),
    #text=atividades.loc[atividades['ano']=='2014', 'data_inicio'].dt.month.value_counts().sort_index(),
    textposition = 'auto',
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

trace2 = go.Bar(
    name='2015',
    x=x,
    y=atividades.loc[atividades['ano']=='2015', 'data_inicio'].dt.month.value_counts().sort_index(),
    #text=atividades.loc[atividades['ano']=='2015', 'data_inicio'].dt.month.value_counts().sort_index(),
    textposition = 'auto',
    marker=dict(
        color='rgb(58,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

trace3 = go.Bar(
    name='2016',
    x=x,
    y=atividades.loc[atividades['ano']=='2016', 'data_inicio'].dt.month.value_counts().sort_index(),
    #text=atividades.loc[atividades['ano']=='2016', 'data_inicio'].dt.month.value_counts().sort_index(),
    textposition = 'auto',
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

trace4 = go.Bar(
    name='2017',
    x=x,
    y=atividades.loc[atividades['ano']=='2017', 'data_inicio'].dt.month.value_counts().sort_index(),
    #text=atividades.loc[atividades['ano']=='2017', 'data_inicio'].dt.month.value_counts().sort_index(),
    textposition = 'auto',
    marker=dict(
        color='rgb(58,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

data = [trace1,trace2, trace3, trace4]
layout = go.Layout(
    title='Atividades de Extensão 2014 - 2017',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='atividades_mes')

In [59]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": atividades['tipo_atividade'].value_counts().values,
      "labels": atividades['tipo_atividade'].value_counts().index,
      "domain": {"x": [0, .48]},
      "name": "Tipo",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Atividades de Extensão 2014 - 2017",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Atividade",
                "x": 0.20,
                "y": 0.5
            }
        ],
      'showlegend': True
    }
}
py.iplot(fig, filename='atividades_extensao_tipo')

In [57]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Histogram(
    x=(atividades['data_fim'] - atividades['data_inicio']).dt.days,
    opacity=0.6
)

data = [trace]
layout = go.Layout(
    title='Número de Dias de Execução das Atividades de Extensão 2014 - 2017',
    xaxis=dict(
        title='Número de Dias'
    ),
    yaxis=dict(
        title='Quantidade de Atividades'
    ),
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='atividades_dias')

In [67]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Histogram(
    x=atividades['publico_interno'],
    opacity=0.6,
    xbins=dict(
        start=0,
        end=800,
        size=10
    ),
)

data = [trace]
layout = go.Layout(
    title='Número de Participantes nas Atividades de Extensão 2014 - 2017',
    xaxis=dict(
        title='Número de Participantes'
    ),
    yaxis=dict(
        title='Quantidade de Atividades'
    ),
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='atividades_publico')